In [1]:
import pandas as pd
# from bokeh.plotting import figure, show, output_notebook # output_file

In [47]:
fpath = r'C:\Users\WB514197\WBG\EEAPV Pacific Observatory Files - Geospatial and Big Data\AIS\all-port-statistics\output\Trade Estimation All.csv'
# fpath = r'C:\Users\WB514197\WBG\EEAPV Pacific Observatory Files - Geospatial and Big Data\AIS\all-port-statistics\output\Trade Estimation All.xlsx'
df = pd.read_csv(fpath)
# df = pd.read_excel(fpath, 1)

In [48]:
df["arrival_dt_pos_utc"] = pd.to_datetime(df["arrival_dt_pos_utc"])
df.loc[:, "year"] = df["arrival_dt_pos_utc"].dt.year
df.loc[:, "month"] = df["arrival_dt_pos_utc"].dt.month
df['iso_week'] = df['arrival_dt_pos_utc'].dt.strftime('%Y-%W')

In [49]:
# df.head(2).to_csv('test.csv')

## Port Arrivals

In [58]:
group_cols = ['iso_week', 'Country', 'Port', 'ShipTypeLevel2']
grouped = df.groupby(group_cols)['vessel_name'].count().unstack().fillna(0)
grouped.reset_index(inplace=True)
grouped.loc[:, "location"] = grouped["Country"] + ", " + grouped["Port"]

In [59]:
country = df.groupby(['iso_week', 'Country', 'ShipTypeLevel2'])['vessel_name'].count().unstack().fillna(0)
country.reset_index(inplace=True)
country.loc[:, "location"] = country["Country"] + "  "

In [60]:
pacific = df.groupby(['iso_week', 'ShipTypeLevel2'])['vessel_name'].count().unstack().fillna(0)
pacific.reset_index(inplace=True)
pacific.loc[:, "Country"] = "All Pacific"
pacific.loc[:, "location"] = "All Pacific"

In [61]:
df_concat = pd.concat([grouped, country, pacific], axis=0)

In [62]:
df_concat.sort_values(by=['location', 'iso_week'], inplace=True)

In [64]:
df_concat['date'] = pd.to_datetime(df_concat['iso_week'] + '-0', format='%Y-%W-%w')

In [65]:
df_concat[['location', 'date', 'Bulk Carriers', 'Dry Cargo/Passenger', 'Tankers']].to_csv('grouped.csv', index=False)

## Trade

In [93]:
df.loc[:, "trade_flow_final_abs"] = df["trade_flow_final"].abs()
df.loc[:, "ym"] = df["year"].astype(str) + "-" + df["month"].astype(str).str.zfill(2)
df.loc[:, "Type"] = df.apply(
    lambda x: "Exports" if x["trade_flow_final"] > 0 else "Imports", axis=1
)

In [107]:
group_cols = ['ym', 'Country', 'Port', 'ShipTypeLevel2', 'Type']
data_cols = ['trade_flow_final_abs']
grouped = df.groupby(group_cols)[data_cols].sum().reset_index()
grouped.loc[:, "location"] = grouped["Country"] + ", " + grouped["Port"]

In [108]:
country = df.groupby(['ym', 'Country', 'ShipTypeLevel2', 'Type'])[data_cols].sum().reset_index()
country.loc[:, "location"] = country["Country"] + "  "

In [109]:
pacific = df.groupby(['ym', 'Country', 'ShipTypeLevel2', 'Type'])[data_cols].sum().reset_index()
pacific.loc[:, "Country"] = "All Pacific"
pacific.loc[:, "location"] = "All Pacific"

In [110]:
df_concat = pd.concat([grouped, country, pacific], axis=0)
df_concat.sort_values(by=['location', 'ym'], inplace=True)

In [112]:
df_concat = df_concat.pivot_table(index=["location", "ym", "Type"], columns='ShipTypeLevel2', values='trade_flow_final_abs').reset_index().fillna(0)

In [ ]:
# df_concat = df_concat.pivot_table(index=["location", "ym"], columns='Type', values='trade_flow_final_abs').reset_index().fillna(0)

In [114]:
df_concat.to_csv('trade-volume-monthly-by-type.csv', index=False)

In [101]:
df_concat.to_csv('trade-volume-monthly.csv', index=False)